# Cell Image augmentation
## Imports

In [ ]:
import cv2
import numpy as np
import os
import albumentations as A

from config import *
from CellProcessor import read_yolo_labels

## Augumentations

In [ ]:
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.2, brightness_limit=0.3, contrast_limit=0.15),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.4, label_fields=['class_labels']),
additional_targets={'phase_mask' : 'mask'})

## Make dirs

In [ ]:
AUGUMENTED_LABELED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Labels_phase_aug/"
AUGUMENTED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Phase_aug/"
AUGUMENTED_MASKS_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Masks_phase_aug/"

try:
    os.mkdir(AUGUMENTED_LABELED_IMAGES_DIR_PATH)
except OSError:
    pass

try:
    os.mkdir(AUGUMENTED_IMAGES_DIR_PATH)
except OSError:
    pass

try:
    os.mkdir(AUGUMENTED_MASKS_DIR_PATH)
except OSError:
    pass

## for every image annotation mask do augumentation

In [ ]:
LABELED_IMAGES_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Labeles_phase/"
IMAGES_MASKS_DIR_PATH = IMAGES_PATH + CELL_TYPE + "_Masks_phase/"
IMAGES_PHASE_PATH = IMAGES_PATH + CELL_TYPE + "_Crop_phase/"

images = sorted(os.listdir(IMAGES_PHASE_PATH))
len_images = len(images)
curr_image_num = 1

for image in images:
    print(str(curr_image_num) + "/" + str(len_images) + "-" + image)
    curr_image_num+=1
    base_image_path = IMAGES_PHASE_PATH + image
    annotation_path = LABELED_IMAGES_DIR_PATH + image.split(".")[0] + ".txt"
    mask_image_path = IMAGES_MASKS_DIR_PATH + image
    if os.path.exists(annotation_path):
        img_base = cv2.imread(base_image_path, cv2.IMREAD_UNCHANGED)
        img_mask = cv2.imread(mask_image_path, cv2.IMREAD_UNCHANGED)
        bboxes, class_labels = read_yolo_labels(annotation_path)
        transformed = transform(image=img_base, bboxes=bboxes, class_labels=class_labels, phase_mask=img_mask)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']
        transformed_mask = transformed['phase_mask']

        if len(transformed_bboxes):
            label_boxes = []
            image_file = AUGUMENTED_IMAGES_DIR_PATH + image
            aug_mask_file = AUGUMENTED_MASKS_DIR_PATH + image
            cv2.imwrite(image_file, transformed_image)
            cv2.imwrite(aug_mask_file, transformed_mask)
            for i in range(0, len(transformed_bboxes)):
                label_box = str(transformed_class_labels[i]) + " " + str(transformed_bboxes[i][0]) + " " + str(transformed_bboxes[i][1]) + " " + str(transformed_bboxes[i][2]) + " " + str(transformed_bboxes[i][3]) + "\n"
                label_boxes.append(label_box)
            label_file = AUGUMENTED_LABELED_IMAGES_DIR_PATH + image.split(".")[0] + ".txt"
            with open(label_file, "w") as ann:
                for lb in label_boxes:
                    ann.write(lb)